In [6]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "mnist"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:5000"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

#from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

In [7]:
from __future__ import annotations

import os
from pathlib import Path
from posixpath import join as urljoin
from typing import Any
from urllib.parse import urlparse, urlunparse
import inspect
import requests
from typing import Any, cast

import structlog
from structlog.stdlib import BoundLogger

LOGGER: BoundLogger = structlog.stdlib.get_logger()

class APIConnectionError(Exception):
    '''Class for connection errors'''
class StatusCodeError(Exception):
    '''Class for status code errors'''
class JSONDecodeError(Exception):
    '''Class for JSON decode errors'''

def create_data_dict(**kwargs):
    return kwargs

def debug(url, data=None):
    LOGGER.debug("Request made.", url=url, data=data)

def get(session, endpoint, *features):
    debug(urljoin(endpoint, *features))
    return make_request(session, 'get', endpoint, None, *features)
def post(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    return make_request(session, 'post', endpoint, data, *features)
def delete(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    return make_request(session, 'delete', endpoint, data, *features)
def put(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    return make_request(session, 'put', endpoint, data, *features)


def make_request(session, method_name, endpoint, data, *features):
    url = urljoin(endpoint, *features)
    method = getattr(session, method_name)
    try: 
        if (data):
            response = method(url, json=data)
        else:
            response = method(url)
        if response.status_code is not 200:
            raise StatusCodeError() 
        json = response.json()
    except (requests.ConnectionError, StatusCodeError, requests.JSONDecodeError) as e:
        handle_error(session, url, method_name.upper(), data, response, e) 
    return json

def handle_error(session, url, method, data, response, error):
    if (type(error) is requests.ConnectionError):
        restapi = os.environ["DIOPTRA_RESTAPI_URI"]
        message = f"Could not connect to the REST API. Is the server running at {restapi}?"
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise APIConnectionError(message);
    if (type(error) is StatusCodeError):
        message = f"Error code {response.status_code} returned."
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise StatusCodeError(message);
    if (type(error) is requests.JSONDecodeError):
        message = f"JSON response could not be decoded."
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise JSONDecodeError(message);

        

class DioptraClient(object):
    def __init__(self, session, address=None, api_version="v1") -> None:
        self._session = session
        self._users = UsersClient(session, "users", address, api_version)
        self._auth = AuthClient(session, "auth", address, api_version)
        self._queues = QueuesClient(session, "queues", address, api_version)
        self._groups = QueuesClient(session, "groups", address, api_version)
    @property
    def users(self):
        return self.get_endpoint(self._users)
    @property
    def auth(self):
        return self.get_endpoint(self._auth)
    @property
    def queues(self):
        return self.get_endpoint(self._queues)
    @property
    def groups(self):
        return self.get_endpoint(self._groups)

    
    def get_endpoint(self, ep):
        ep.session = self._session
        return ep

class Endpoint(object):
    def __init__(self, session, ep_name, address, api_version) -> None:
        address = (
            f"{address}/api/{api_version}"
            if address
            else f"{os.environ['DIOPTRA_RESTAPI_URI']}/api/{api_version}"
        )
        self._scheme, self._netloc, self._path, _, _, _ = urlparse(address)
        self._ep_name = ep_name
        self._session = session
    @property
    def session(self):
        return self._session
    @session.setter
    def session(self, s):
        self._session = s
    @property
    def url(self):
        return self.def_endpoint(self._ep_name)
    def def_endpoint(self, name):
        '''creates base url for an endpoint by name'''
        return urlunparse(
            (self._scheme, self._netloc, urljoin(self._path, name + "/"), "", "", "")
        )

class UsersClient(Endpoint):
    def get_all(self):
        '''gets all users'''
        return get(self.session, self.url)
    def create(self, username, email, password, confirm_password):
        '''creates a user'''
        d = {"username":username,
             "email": email,
             "password": password,
             "confirmPassword": confirm_password}
        return post(self.session, self.url, d)
    def get_by_id(self, id):
        '''get a user by id'''
        return get(self.session, self.url, str(id))
    def update_password_by_id(self, id, old_password, new_password, confirm_new_password):
        '''change a user's password by id'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, str(id), 'password')
    def current(self):
        '''get the current user'''
        return get(self.session, self.url, 'current')
    def delete_current(self, password):
        '''delete the current user'''
        d = {"password":password}
        return delete(self.session, self.url, d, 'current')
    def modify_current(self, username, email):
        '''modify the current user'''
        d = {"username":username,
             "email": email}
        return put(self.session, self.url, d, 'current')
    def modify_current_password(self, old_password, new_password, confirm_new_password):
        '''modify the current user's password'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, 'current', 'password')
    def failed_user_post(self):
        '''create a post request with an invalid schema, for testing'''
        return post(self.session, self.url, {'a':'doesnotexist'})
    def failed_user_get(self):
        '''create a get request to an invalid url, for testing'''
        return get(self.session, self.url, 'doesnotexist')

class AuthClient(Endpoint):
    def login(self, username, password):
        '''login as the given user'''
        d = {"username":username,
             "password": password}
        return post(self.session, self.url, d, 'login')
    def logout(self, everywhere):
        '''logout as the current user'''
        d = {"everywhere": everywhere}
        return post(self.session, self.url, d, 'logout')

class GroupsClient(Endpoint):
    def get_all(self):
        '''get all groups'''
        return get(self.session, self.url)
    def get_by_id(self, gid):
        '''get a group by id'''
        return get(self.session, self.url, str(gid))
        
class QueuesClient(Endpoint):
    def get_all(self):
        '''gets all queues'''
        return get(self.session, self.url)
    def create(self, group, name, description):
        '''create a queue'''
        d = {"group": group,
             "name": name,
             "description": description
            }
        return post(self.session, self.url, d)
    def get_drafts(self):
        '''gets the queue drafts'''
        return get(self.session, self.url, 'drafts')
    def create_draft(self, group, name, description):
        '''create a queue'''
        d = {"group": group,
             "name": name,
             "description": description
            }
        return post(self.session, self.url, d, 'drafts')
    def modify_draft_by_draft_id(self, draft_id, name, description):
        '''modify a queue draft by id'''
        d = {"name": name,
             "description": description
            }
        return put(self.session, self.url, d, 'drafts', str(draft_id))
    def delete_draft_by_draft_id(self, draft_id):
        '''delete a queue draft by id'''
        d = None
        return delete(self.session, self.url, d, 'drafts', str(draft_id))
    def get_draft_by_draft_id(self, draft_id):
        '''get a queue draft by id'''
        return get(self.session, self.url, 'drafts', str(draft_id))
    def modify_by_queue_id(self, queue_id, name, description):
        '''modify a queue by id'''
        d = {"name": name,
             "description": description
            }
        return put(self.session, self.url, d, str(queue_id))
    def delete_by_queue_id(self, queue_id):
        '''delete a queue by id'''
        d = None
        return delete(self.session, self.url, d, str(queue_id))
    def get_by_queue_id(self, queue_id):
        '''get a queue by id'''
        return get(self.session, self.url, str(queue_id))
    def modify_draft_by_queue_id(self, queue_id, name, description):
        '''modify a queue by id'''
        d = {"name": name,
             "description": description
            }
        return put(self.session, self.url, d, str(queue_id), 'draft')
    def create_draft_by_queue_id(self, queue_id, name, description):
        '''create a queue by id'''
        d = {"name": name,
             "description": description
            }
        return post(self.session, self.url, d, str(queue_id), 'draft')
    def delete_draft_by_queue_id(self, queue_id):
        '''delete a queue by id'''
        d = None
        return delete(self.session, self.url, d, str(queue_id), 'draft')
    def get_draft_by_queue_id(self, queue_id):
        '''get a queue by id'''
        return get(self.session, self.url, str(queue_id), 'draft')
    


In [8]:
client = DioptraClient(requests.Session())

In [9]:
client.users.create('testuser','testuser@gmail.com','testuserpassword','testuserpassword')

2024-06-20 14:19:05 [debug    ] Request made.                  data={'username': 'testuser', 'email': 'testuser@gmail.com', 'password': 'testuserpassword', 'confirmPassword': 'testuserpassword'} url=http://localhost:5000/api/v1/users/


{'username': 'testuser',
 'email': 'testuser@gmail.com',
 'id': 1,
 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}],
 'createdOn': '2024-06-20T18:19:05.229207+00:00',
 'lastModifiedOn': '2024-06-20T18:19:05.229207+00:00',
 'lastLoginOn': None,
 'passwordExpiresOn': '2025-06-20T18:19:05.229207+00:00'}

In [10]:
client.users.create('testuser2','testuser2@gmail.com','testuserpassword','testuserpassword')

2024-06-20 14:19:06 [debug    ] Request made.                  data={'username': 'testuser2', 'email': 'testuser2@gmail.com', 'password': 'testuserpassword', 'confirmPassword': 'testuserpassword'} url=http://localhost:5000/api/v1/users/


{'username': 'testuser2',
 'email': 'testuser2@gmail.com',
 'id': 2,
 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}],
 'createdOn': '2024-06-20T18:19:06.197123+00:00',
 'lastModifiedOn': '2024-06-20T18:19:06.197123+00:00',
 'lastLoginOn': None,
 'passwordExpiresOn': '2025-06-20T18:19:06.197123+00:00'}

In [11]:
client.auth.login('testuser','testuserpassword')

2024-06-20 14:19:06 [debug    ] Request made.                  data={'username': 'testuser', 'password': 'testuserpassword'} url=http://localhost:5000/api/v1/auth/login


{'username': 'testuser', 'status': 'Login successful'}

In [12]:
client.users.get_all()

2024-06-20 14:19:06 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/users/


{'index': 0,
 'isComplete': True,
 'totalNumResults': 2,
 'first': '/api/v1/users?index=0&pageLength=10',
 'data': [{'username': 'testuser', 'email': 'testuser@gmail.com', 'id': 1},
  {'username': 'testuser2', 'email': 'testuser2@gmail.com', 'id': 2}]}

In [13]:
client.users.get_by_id('1')

2024-06-20 14:19:07 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/users/1


{'username': 'testuser', 'email': 'testuser@gmail.com', 'id': 1}

In [14]:
client.users.get_by_id('2')

2024-06-20 14:19:07 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/users/2


{'username': 'testuser2', 'email': 'testuser2@gmail.com', 'id': 2}

In [15]:
client.users.current()

2024-06-20 14:19:08 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/users/current


{'username': 'testuser',
 'email': 'testuser@gmail.com',
 'id': 1,
 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}],
 'createdOn': '2024-06-20T18:19:05.229207+00:00',
 'lastModifiedOn': '2024-06-20T18:19:05.229207+00:00',
 'lastLoginOn': '2024-06-20T18:19:06.669522+00:00',
 'passwordExpiresOn': '2025-06-20T18:19:05.229207+00:00'}

In [16]:
client.users.update_password_by_id(client.users.current()['id'],'testuserpassword','newtestuserpassword','newtestuserpassword')

2024-06-20 14:19:08 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/users/current
2024-06-20 14:19:08 [debug    ] Request made.                  data={'oldPassword': 'testuserpassword', 'newPassword': 'newtestuserpassword', 'confirmNewPassword': 'newtestuserpassword'} url=http://localhost:5000/api/v1/users/1/password


{'status': 'Password Change Success'}

In [17]:
client.users.modify_current('testuser','newemail@email.com')

2024-06-20 14:19:10 [debug    ] Request made.                  data={'username': 'testuser', 'email': 'newemail@email.com'} url=http://localhost:5000/api/v1/users/current
2024-06-20 14:19:10 [error    ] Error code 401 returned.       data={'username': 'testuser', 'email': 'newemail@email.com'} method=PUT response={"message": "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}
 url=http://localhost:5000/api/v1/users/current


StatusCodeError: Error code 401 returned.

In [18]:
client.auth.login('testuser','newtestuserpassword')

2024-06-20 14:19:11 [debug    ] Request made.                  data={'username': 'testuser', 'password': 'newtestuserpassword'} url=http://localhost:5000/api/v1/auth/login


{'username': 'testuser', 'status': 'Login successful'}

In [19]:
client.users.modify_current('testuser','newemail@email.com')

2024-06-20 14:19:21 [debug    ] Request made.                  data={'username': 'testuser', 'email': 'newemail@email.com'} url=http://localhost:5000/api/v1/users/current


{'username': 'testuser',
 'email': 'newemail@email.com',
 'id': 1,
 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}],
 'createdOn': '2024-06-20T18:19:05.229207+00:00',
 'lastModifiedOn': '2024-06-20T18:19:21.879001+00:00',
 'lastLoginOn': '2024-06-20T18:19:11.354018+00:00',
 'passwordExpiresOn': '2025-06-20T18:19:08.785725+00:00'}

In [20]:
client.users.modify_current_password('newtestuserpassword','newnewtestuserpassword','newnewtestuserpassword')

2024-06-20 14:19:22 [debug    ] Request made.                  data={'oldPassword': 'newtestuserpassword', 'newPassword': 'newnewtestuserpassword', 'confirmNewPassword': 'newnewtestuserpassword'} url=http://localhost:5000/api/v1/users/current/password


{'status': 'Password Change Success'}

In [21]:
client.auth.logout(True)

2024-06-20 14:19:22 [debug    ] Request made.                  data={'everywhere': True} url=http://localhost:5000/api/v1/auth/logout
2024-06-20 14:19:22 [error    ] Error code 401 returned.       data={'everywhere': True} method=POST response={"message": "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}
 url=http://localhost:5000/api/v1/auth/logout


StatusCodeError: Error code 401 returned.

In [22]:
client.auth.login('testuser','newnewtestuserpassword')

2024-06-20 14:19:24 [debug    ] Request made.                  data={'username': 'testuser', 'password': 'newnewtestuserpassword'} url=http://localhost:5000/api/v1/auth/login


{'username': 'testuser', 'status': 'Login successful'}

In [23]:
client.users.delete_current('newnewtestuserpassword')

2024-06-20 14:19:34 [debug    ] Request made.                  data={'password': 'newnewtestuserpassword'} url=http://localhost:5000/api/v1/users/current


{'status': 'Success'}

In [24]:
client.users.failed_user_post() # if we get the schema wrong, it does return JSON, raise if not 200, but send json back to logs

2024-06-20 14:19:34 [debug    ] Request made.                  data={'a': 'doesnotexist'} url=http://localhost:5000/api/v1/users/
2024-06-20 14:19:35 [error    ] Error code 400 returned.       data={'a': 'doesnotexist'} method=POST response={"schema_errors": {"a": ["Unknown field."]}}
 url=http://localhost:5000/api/v1/users/


StatusCodeError: Error code 400 returned.

In [25]:
client.users.failed_user_get() # if we get the URL wrong, it does not return JSON

2024-06-20 14:19:35 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/users/doesnotexist
2024-06-20 14:19:35 [error    ] Error code 404 returned.       data=None method=GET response=<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
 url=http://localhost:5000/api/v1/users/doesnotexist


StatusCodeError: Error code 404 returned.

In [26]:
try:
    client.users.create('queuemgr','queuemgr@dioptra.nccoe.nist.gov','lmnopQUEUErs','lmnopQUEUErs')
except:
    pass # ignore if user exists already
client.auth.login('queuemgr','lmnopQUEUErs')

2024-06-20 14:19:37 [debug    ] Request made.                  data={'username': 'queuemgr', 'email': 'queuemgr@dioptra.nccoe.nist.gov', 'password': 'lmnopQUEUErs', 'confirmPassword': 'lmnopQUEUErs'} url=http://localhost:5000/api/v1/users/
2024-06-20 14:19:37 [debug    ] Request made.                  data={'username': 'queuemgr', 'password': 'lmnopQUEUErs'} url=http://localhost:5000/api/v1/auth/login


{'username': 'queuemgr', 'status': 'Login successful'}

In [27]:
cpu_id = client.queues.create(1, 'tensorflow_cpu2', 'for running tensorflow on a cpu')['id']

2024-06-20 14:19:38 [debug    ] Request made.                  data={'group': 1, 'name': 'tensorflow_cpu2', 'description': 'for running tensorflow on a cpu'} url=http://localhost:5000/api/v1/queues/


In [28]:
client.queues.modify_by_queue_id(cpu_id, 'tensorflow_cpu2', 'for running tensorflow on a cpu - modified')

2024-06-20 14:19:38 [debug    ] Request made.                  data={'name': 'tensorflow_cpu2', 'description': 'for running tensorflow on a cpu - modified'} url=http://localhost:5000/api/v1/queues/1


{'id': 1,
 'snapshot': 2,
 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
 'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
 'createdOn': '2024-06-20T18:19:38.580676+00:00',
 'lastModifiedOn': '2024-06-20T18:19:38.580676+00:00',
 'latestSnapshot': True,
 'tags': [],
 'name': 'tensorflow_cpu2',
 'description': 'for running tensorflow on a cpu - modified'}

In [29]:
client.queues.get_all()

2024-06-20 14:19:39 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/queues/


{'index': 0,
 'isComplete': True,
 'totalNumResults': 1,
 'first': '/api/v1/queues?index=0&pageLength=10',
 'data': [{'id': 1,
   'snapshot': 2,
   'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
   'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
   'createdOn': '2024-06-20T18:19:38.580676+00:00',
   'lastModifiedOn': '2024-06-20T18:19:38.580676+00:00',
   'latestSnapshot': True,
   'tags': [],
   'name': 'tensorflow_cpu2',
   'description': 'for running tensorflow on a cpu - modified'}]}

In [30]:
gpu_draft_id = client.queues.create_draft(1, 'tensorflow_gpu', 'for running tensorflow on a gpu')['id']

2024-06-20 14:19:39 [debug    ] Request made.                  data={'group': 1, 'name': 'tensorflow_gpu', 'description': 'for running tensorflow on a gpu'} url=http://localhost:5000/api/v1/queues/drafts


In [31]:
client.queues.modify_draft_by_draft_id(gpu_draft_id, 'tensorflow_gpu', 'for running tensorflow on a gpu - modified')

2024-06-20 14:19:39 [debug    ] Request made.                  data={'name': 'tensorflow_gpu', 'description': 'for running tensorflow on a gpu - modified'} url=http://localhost:5000/api/v1/queues/drafts/1


{'id': 1,
 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
 'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
 'createdOn': '2024-06-20T18:19:39.412053+00:00',
 'lastModifiedOn': '2024-06-20T18:19:39.790130+00:00',
 'resourceType': 'queue',
 'payload': {'name': 'tensorflow_gpu',
  'description': 'for running tensorflow on a gpu - modified'}}

In [32]:
client.queues.get_drafts()

2024-06-20 14:19:40 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/queues/drafts


{'index': 0,
 'isComplete': True,
 'totalNumResults': 1,
 'first': '/api/v1/queues/drafts?index=0&pageLength=10',
 'data': [{'id': 1,
   'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
   'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
   'createdOn': '2024-06-20T18:19:39.412053+00:00',
   'lastModifiedOn': '2024-06-20T18:19:39.790130+00:00',
   'resourceType': 'queue',
   'payload': {'name': 'tensorflow_gpu',
    'description': 'for running tensorflow on a gpu - modified'}}]}

In [33]:
client.queues.get_draft_by_draft_id(gpu_draft_id)

2024-06-20 14:19:40 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/queues/drafts/1


{'id': 1,
 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
 'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
 'createdOn': '2024-06-20T18:19:39.412053+00:00',
 'lastModifiedOn': '2024-06-20T18:19:39.790130+00:00',
 'resourceType': 'queue',
 'payload': {'name': 'tensorflow_gpu',
  'description': 'for running tensorflow on a gpu - modified'}}

In [34]:
client.queues.delete_draft_by_draft_id(gpu_draft_id)

2024-06-20 14:19:40 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/queues/drafts/1


{'status': 'Success'}

In [35]:
client.queues.create_draft_by_queue_id(cpu_id, 'tensorflow_cpu2', 'oops my draft deleted the description')

2024-06-20 14:19:41 [debug    ] Request made.                  data={'name': 'tensorflow_cpu2', 'description': 'oops my draft deleted the description'} url=http://localhost:5000/api/v1/queues/1/draft


{'id': 1,
 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
 'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
 'createdOn': '2024-06-20T18:19:41.380028+00:00',
 'lastModifiedOn': '2024-06-20T18:19:41.380028+00:00',
 'resourceType': 'queue',
 'payload': {'name': 'tensorflow_cpu2',
  'description': 'oops my draft deleted the description'},
 'resource': 1,
 'resourceSnapshot': 2,
 'numOtherDrafts': 0}

In [36]:
client.queues.get_draft_by_queue_id(cpu_id)

2024-06-20 14:19:41 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/queues/1/draft


{'id': 1,
 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
 'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
 'createdOn': '2024-06-20T18:19:41.380028+00:00',
 'lastModifiedOn': '2024-06-20T18:19:41.380028+00:00',
 'resourceType': 'queue',
 'payload': {'name': 'tensorflow_cpu2',
  'description': 'oops my draft deleted the description'},
 'resource': 1,
 'resourceSnapshot': 2,
 'numOtherDrafts': 0}

In [37]:
client.queues.modify_draft_by_queue_id(cpu_id, 'tensorflow_cpu2', 'wait go back')

2024-06-20 14:19:41 [debug    ] Request made.                  data={'name': 'tensorflow_cpu2', 'description': 'wait go back'} url=http://localhost:5000/api/v1/queues/1/draft


{'id': 1,
 'group': {'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'},
 'user': {'id': 3, 'username': 'queuemgr', 'url': '/api/v1/users/3'},
 'createdOn': '2024-06-20T18:19:41.380028+00:00',
 'lastModifiedOn': '2024-06-20T18:19:41.955400+00:00',
 'resourceType': 'queue',
 'payload': {'name': 'tensorflow_cpu2', 'description': 'wait go back'},
 'resource': 1,
 'resourceSnapshot': 2,
 'numOtherDrafts': 0}

In [38]:
client.queues.delete_draft_by_queue_id(cpu_id)

2024-06-20 14:19:42 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/queues/1/draft


{'status': 'Success'}

In [39]:
client.queues.delete_by_queue_id(cpu_id)

2024-06-20 14:19:42 [debug    ] Request made.                  data=None url=http://localhost:5000/api/v1/queues/1


{'status': 'Success'}